In [ ]:
from openai import OpenAI
from collections import Counter
from natsort import natsorted
import requests
from requests.exceptions import Timeout, RequestException

model_name = "qwen2.5:72b"
client = OpenAI(
    base_url='http://kt-gpu5.ijs.si:11435/v1',
    api_key='ollama',  # required, but unused
)

def create_chat_completions(model_name, message, timeout=100):
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": message},
            ],
            timeout=timeout
        )
        return response.choices[0].message.content
    except Timeout:
        return None
    except RequestException as e:
        return None